#web scraping thingy 

In [54]:
# ["type", "beds", "price", "location", "url", "aquisition", "contact", "duration", "furnished", "serviced", "imageurl"] //compare rent vs buy(month v day), should I buy/recommeded
# paginationSelector = soup.find('div', class_='pagination-area').nav.ul.findAll('li')[3].a['href']
# paginationSelector

'''
# return all div elements with a class attribute containing the text 'listing-col-':
for EachPart in soup.select('div[class*="listing-col-"]'):
    print EachPart.get_text()

'''


'\n# return all div elements with a class attribute containing the text \'listing-col-\':\nfor EachPart in soup.select(\'div[class*="listing-col-"]\'):\n    print EachPart.get_text()\n\n'

In [55]:
# !pip install selenium
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from bs4.element import Comment
import re
import csv
import json
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import numpy as np


In [56]:
url = """

https://www.smartcontractresearch.org/latest


"""



In [60]:
def getPageData(url):
    driver = webdriver.Chrome('./chromedriver') 
    driver.get(url)
    time.sleep(10)     
    SCROLL_PAUSE_TIME = 10
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
    # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    result = []
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')   
    scrf_summaries =  soup.find_all("tr")
    for post in scrf_summaries:
        if post.find('a', href=True):
            post_url = post.find('a', href=True)['href']
            if post_url.startswith('/'):
                post_url = 'https://www.smartcontractresearch.org' + post_url
            result.append(post_url)

    driver.close()
    return result

    
result =  getPageData(url)


/var/folders/yt/r5vqk9zd7f35c460q8mztp2w0000gn/T/ipykernel_7316/2375564607.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


In [ ]:
##UTILITIES

# WRITE DATA TO CSV FILE
# data is a list of lists

def createCsvFile(filename, rose):
    header =[ "title", "url", "category", "total_likes", "paper link", "tags", "comments"]
    with open(filename, 'w') as f:
        write = csv.writer(f)
        write.writerow(header)
        write.writerows(rose)

def unique(list1):
    unique_list = []
    for x in list1:
        if x not in unique_list:
            unique_list.append(x)
    return unique_list
  

In [ ]:
result = unique(result)
len(result)


120

In [61]:
finalPosts = []
header =[ "title", "url", "category", "total_likes", "paper link", "tags", "comments"]

def getPostComments(url):
    driver = webdriver.Chrome('./chromedriver') 
    result = []
    driver.get(url)
    time.sleep(10) 
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser') 
    # classes = soup.find_all(class_=True)
    category = soup.find(class_='category-name').text or ''
    tags = soup.find_all("a", class_='discourse-tag') or [""]
    tags = [tag.text for tag in tags]  
    tags = unique(tags)
    title= soup.find(class_='fancy-title').text.replace('\n', '')  or ''
    comments = soup.find_all("div", class_='cooked') or ['']
    comments = [comment.text for comment in comments]
    comments = [comment.replace('\n', '') for comment in comments] 
    total_likes = soup.find_all(class_='number')[3].text or ""
    links = soup.find_all('a', href=True) or [""]
    returnedLink = []
    for link in links:
        if "citation" in link['href']:
            index = links.index(link)
            returnedLink.append(links[index + 1]["href"]) 

    if len(comments) - 1 >= 10:
        finalPosts.append([ title, url, category, total_likes, unique(returnedLink), tags, comments])
        
    driver.close()

In [ ]:
for post in result:
    # getPostComments(post[0])
    try:
        getPostComments(post)
        print(result.index(post), " of ", len(result))
        print(finalPosts[-1][0:2])
        
    except Exception as e:
        print("error on " + post)
        print(e)


createCsvFile('scrfabove10.csv', finalPosts)

        

In [63]:
len(finalPosts)

59

In [10]:
# WRITE DATA TO CSV FILE
resultHeader = ["url", "title"]
createCsvFile('scrfabove10.csv', resultHeader, result)

In [6]:
propertyForSale = pd.read_csv("ScrapedSalePropertyDataSet.csv")
propertyForRent = pd.read_csv("ScrapedRentalDataSet.csv")

In [78]:

def abstractedReadToDataFrame(filename):
    df = pd.read_csv(filename)
    df = df.dropna()
    df = df.reset_index(drop=True)
    return df

df = abstractedReadToDataFrame("scrfabove10.csv")
df.sample(20)
df.sort_values(by=["total_likes"], ascending=False) 

# where title starts with "A"
# df[df['title'].str.startswith("R")].sort_values(by=["total_likes"], ascending=False)
df[df['title'].str.contains("r")]
# print title for all rows
df['title'].values

# strip whitespace from title
df['title'] = df['title'].str.strip()



In [100]:
# df['title'].values
# df['title'].str.contains("research")
# convert title to string
# df['title'] = df['title'].astype(str)
from tabnanny import check


summaries_only = df[df['url'].str.contains("summary")]  

sorted_summaries = summaries_only.sort_values(by=["total_likes"], ascending=False)
sorted_summaries
whereTagsShort = sorted_summaries[sorted_summaries['tags'].str.len() == 2]
whereTagsShort


,title,url,category,total_likes,paper link,tags,comments


In [109]:
from ast import literal_eval

try:
    sorted_summaries['tags'] = sorted_summaries['tags'].apply(literal_eval)
except:
    pass


In [110]:
sorted_summaries['tags'].iloc[0]

['scalability', 'iot', 'summary']

In [134]:
whereTagsShort = sorted_summaries[sorted_summaries['tags'].str.len() < 3 ]
whereTagsShort = whereTagsShort[whereTagsShort["comments"].str.len() > 10]
whereTagsShort["title"]
for i in whereTagsShort["title"]:
    print(i.replace('Research Summary:', '').strip())


Leveraging Blockchain for Greater Accessibility of Machine Learning Models
SoK: Applying Blockchain Technology in Industrial Internet of Things
On the Economic Design of Stablecoins
Ethereum Name Service: the Good, the Bad, and the Ugly
Research Summary - What is in Your Wallet? Privacy and Security Issues in Web 3.0
Using Distributed Ledger Technologies to Improve and Maximise the Collection of Property Taxes
Towards A first step to understand flash loan and its application in Defi Ecosystem


In [140]:
# print title and url for all rows
for i in range(len(whereTagsShort)):
    print(whereTagsShort["title"].iloc[i].replace('Research Summary:', '').strip(), whereTagsShort["url"].iloc[i])

Leveraging Blockchain for Greater Accessibility of Machine Learning Models https://www.smartcontractresearch.org/t/research-summary-leveraging-blockchain-for-greater-accessibility-of-machine-learning-models/808
SoK: Applying Blockchain Technology in Industrial Internet of Things https://www.smartcontractresearch.org/t/research-summary-sok-applying-blockchain-technology-in-industrial-internet-of-things/866
On the Economic Design of Stablecoins https://www.smartcontractresearch.org/t/research-summary-on-the-economic-design-of-stablecoins/817
Ethereum Name Service: the Good, the Bad, and the Ugly https://www.smartcontractresearch.org/t/research-summary-ethereum-name-service-the-good-the-bad-and-the-ugly/1587
Research Summary - What is in Your Wallet? Privacy and Security Issues in Web 3.0 https://www.smartcontractresearch.org/t/research-summary-what-is-in-your-wallet-privacy-and-security-issues-in-web-3-0/1225
Using Distributed Ledger Technologies to Improve and Maximise the Collection of